In [ ]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq,
    TrainerCallback
)

torch.manual_seed(101)

In [ ]:
EPOCHS = 10
LEARNING_RATE = 1e-4
BATCH_SIZE = 8
OUTPUT_PATH = "./output"
MODEL = "t5-base"
DATA_PATH = '../data/qrels.train.tsv'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Processing MS-MARCO dataset.

train_samples = []
with open(DATA_PATH, 'r', encoding='utf-8') as file:
    for num, line in tqdm(enumerate(file)):
        if num == 0: continue
        if num > 6.4e5 * 10:
            break
        query, positive, negative = line.split("\t")
        train_samples.append((query, positive, 'true'))
        train_samples.append((query, negative, 'false'))

1065523it [00:20, 51334.66it/s]


In [ ]:
# Converting MS-MARCO to torch dataset for fine-tuning


class dataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        text = f'Query: {sample[0]} Document: {sample[1]} Relevant:'
        return {
          'text': text,
          'labels': sample[2],
        }
    
    
dataset_train = dataset(train_samples)

In [ ]:
# Loading the model and last saved checkpoint to resume training

check_point_path = "./output/" + os.listdir("./output/")[-1]

model = AutoModelForSeq2SeqLM.from_pretrained('t5-base').to(device)
tokenizer = AutoTokenizer.from_pretrained('t5-base')

In [ ]:
# Setting training arguments and parameters

train_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    remove_unused_columns=False, 
    output_dir=OUTPUT_PATH,
    do_train=True,
    save_strategy='steps',
    save_steps=100,
    logging_steps=100,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=16,
    learning_rate=LEARNING_RATE,
    weight_decay=5e-5,
    num_train_epochs=EPOCHS,
    warmup_steps=1000,
    adafactor=True,
    seed=1,
    disable_tqdm=False,
    fp16=True,
    load_best_model_at_end=False,
    dataloader_pin_memory=False,
    resume_from_checkpoint=True,
)

/home/pratham/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/training_args.py:1725: FutureWarning: `--adafactor` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--optim adafactor` instead
  warnings.warn(


In [ ]:
# Collating data

def data_collate(batch):
    texts = [example["text"] for example in batch]
    tokenized = tokenizer(
        texts,
        padding=True,
        truncation="longest_first",
        return_tensors="pt",
        max_length=512,
    )

    tokenized["labels"] = tokenizer(
        [example["labels"] for example in batch], return_tensors="pt"
    )["input_ids"]

    for name in tokenized:
        tokenized[name] = tokenized[name].to(device)

    return tokenized

In [10]:
# Resume training from last saved checkpoint

trainer = Seq2SeqTrainer(
    model=model,
    args=train_args,
    train_dataset=dataset_train,
    tokenizer=tokenizer,
    data_collator=data_collate
)

trainer.train(resume_from_checkpoint=check_point_path)

/tmp/ipykernel_13463/3159938339.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].
/home/pratham/miniconda3/envs/torch/lib/python3.12/site-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via 

Step,Training Loss
500,0.263700
600,0.257800


KeyboardInterrupt: 

In [ ]:
# Saving final model

trainer.save_model("../full_tuned_model")
trainer.save_state()